In [2]:
!pip install datasets tokenizers tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.


In [3]:
# Import libraries
from tokenizers import Tokenizer, models, pre_tokenizers, trainers
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from datasets import load_dataset
import os
import torch
import tiktoken

# Check GPU availability
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0))

# Step 1: Load C4 dataset from Hugging Face
def load_c4_data():
    output_file = "c4_sample.txt"
    if not os.path.exists(output_file):
        print("Loading C4 dataset from Hugging Face...")
        # Stream the English subset of C4 dataset
        dataset = load_dataset("allenai/c4", "en", split="train", streaming=True)

        # Write a sample of the dataset to a file (e.g., 100k examples)
        with open(output_file, 'w', encoding='utf-8') as f:
            for i, example in enumerate(dataset):
                if i >= 100000:  # Limit to 100k examples for Colab feasibility
                    break
                f.write(example['text'] + "\n")
        print("C4 sample saved to", output_file)
    return output_file

# Step 2: Define and train the tokenizer
def train_tokenizer(file_path):
    tokenizer = Tokenizer(models.BPE())
    tokenizer.pre_tokenizer = Whitespace()
    # No explicit decoder needed for BPE; decoding is handled internally

    trainer = BpeTrainer(
        vocab_size=30000,  # Adjustable; GPT-4 uses ~100k
        min_frequency=2,
        special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
    )

    print("Training tokenizer...")
    tokenizer.train([file_path], trainer)
    print("Training complete!")

    tokenizer.save("custom_tokenizer.json")
    return tokenizer

# Step 3: Test the tokenizer
def test_tokenizer(tokenizer):
    test_sentence = "The quick brown fox jumps over the lazy dog."
    encoded = tokenizer.encode(test_sentence)

    print("Original:", test_sentence)
    print("Tokens:", encoded.tokens)
    print("Token IDs:", encoded.ids)
    print("Decoded:", tokenizer.decode(encoded.ids))

# Main execution
if __name__ == "__main__":
    # Load C4 data
    text_file = load_c4_data()

    # Train the tokenizer
    tokenizer = train_tokenizer(text_file)

    # Test it
    test_tokenizer(tokenizer)

    # Compare with GPT-like tokenizer
    gpt_tokenizer = tiktoken.get_encoding("cl100k_base")
    test_sentence = "The quick brown fox jumps over the lazy dog."
    gpt_encoded = gpt_tokenizer.encode(test_sentence)
    print("\nGPT Tokenizer Comparison:")
    print("GPT Tokens:", [gpt_tokenizer.decode([t]) for t in gpt_encoded])
    print("GPT Token Count:", len(gpt_encoded))
    print("Custom Token Count:", len(tokenizer.encode(test_sentence).tokens))

GPU Available: True
GPU Name: NVIDIA A100-SXM4-40GB
Loading C4 dataset from Hugging Face...


README.md:   0%|          | 0.00/41.1k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

C4 sample saved to c4_sample.txt
Training tokenizer...
Training complete!
Original: The quick brown fox jumps over the lazy dog.
Tokens: ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.']
Token IDs: [4481, 5668, 9486, 15281, 25743, 4663, 4410, 18541, 6499, 37]
Decoded: The quick brown fox jumps over the lazy dog .

GPT Tokenizer Comparison:
GPT Tokens: ['The', ' quick', ' brown', ' fox', ' jumps', ' over', ' the', ' lazy', ' dog', '.']
GPT Token Count: 10
Custom Token Count: 10
